## LangChain intro

In [1]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

In [2]:
load_dotenv()

True

### LLM model initializing

In [3]:
openai_model = "gpt-4o-mini"

# For normal accurate responses
llm = ChatOpenAI(temperature=0.0, model=openai_model)

# For unique creative responses
creative_llm = ChatOpenAI(temperature=0.9, model=openai_model)

### Prompts

In [4]:
from langchain.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate
)

In [5]:
# Defining the system prompt (how the AI should act)
system_prompt = SystemMessagePromptTemplate.from_template(
    "You are an AI assistant called {name} that helps generate article titles.",
    input_variables=["name"]
)

In [6]:
user_prompt = HumanMessagePromptTemplate.from_template(
    """
    You are tasked with creating a name for a article. 
    The article is here for you to examine:
    ----
    {article}
    ----
    The name should be based of the context of the article. 
    Be creative, but make sure the names are clear, catchy,
    and relevant to the theme of the article. 

    Only output the article name, no other explanations or 
    text can be provided.
    """,
    input_variable=["article"]
)

In [7]:
first_prompt = ChatPromptTemplate.from_messages([system_prompt, user_prompt])

In [8]:
print(first_prompt.format(name="Joe", article="TEST STRING"))

System: You are an AI assistant called Joe that helps generate article titles.
Human: 
    You are tasked with creating a name for a article. 
    The article is here for you to examine:
    ----
    TEST STRING
    ----
    The name should be based of the context of the article. 
    Be creative, but make sure the names are clear, catchy,
    and relevant to the theme of the article. 

    Only output the article name, no other explanations or 
    text can be provided.
    


### Chains
We'll be using LangChain Expression Language (LCEL) to construct our chain. \
The firts dict segment define the inputs, then de '|' indicates the output from the left of the pipe will be fed into the input to the right of the pipe

In [9]:
chain_one = (
    {
        "article": lambda x: x["article"],
        "name": lambda x: x["name"]
    }
    | first_prompt
    | creative_llm
    | {"article_title": lambda x: x.content}
)

In [10]:
article_file = open(r"./article.txt", encoding="utf-8")
article = article_file.read()

In [11]:
article_title_message = chain_one.invoke({"name": "Joe", "article": article})

In [12]:
article_title_message["article_title"]

'"Agents of Change: The Future of AI Through Neuro-Symbolic Systems"'

### Another Text Generation Pipeline

In [13]:
second_system_prompt = SystemMessagePromptTemplate.from_template(
    "You are an AI assistant that helps build good articles."
)

In [14]:
second_user_prompt = HumanMessagePromptTemplate.from_template(
    """
    You are tasked with creating a description for the article. 
    The article is here for you to examine:
    ----
    {article}
    ----
    Here is the article title '{article_title}'
    Output the SEO friendly article description, make 
    sure to dont exceed 150 characters.
    Do not output anything other than the description.
    """,
    input_variable=["article", "article_title"]
)

In [15]:
second_prompt = ChatPromptTemplate.from_messages([
    second_system_prompt,
    second_user_prompt
])

In [16]:
chain_two = (
    {
        "article": lambda x: x["article"],
        "article_title": lambda x: x["article_title"]
    }
    | second_prompt
    | llm
    | {"summary": lambda x: x.content}
)

In [17]:
article_description_message = chain_two.invoke({
    "article": article,
    "article_title": article_title_message["article_title"]
})

In [18]:
article_description_message["summary"]

'Explore the future of AI with neuro-symbolic systems, merging LLMs and code to create powerful agents for real-world applications.'

### Structured Outputs

In [19]:
from pydantic import BaseModel, Field

In [20]:
third_system_prompt = SystemMessagePromptTemplate.from_template(
    "You are an AI assistant that helps build and improve articles."
)

In [21]:
third_user_prompt = HumanMessagePromptTemplate.from_template(
    """
    You are tasked with creating a new paragraph for the
    article. The article is here for you to examine:
    ---
    {article}
    ---
    Choose one paragraph to review and edit. During your edit
    ensure you provide constructive feedback to the user so they
    can learn where to improve their own writing.
    """,
    input_variables=["article"]
)

In [22]:
third_prompt = ChatPromptTemplate.from_messages([
    third_system_prompt,
    third_user_prompt
])

In [23]:
# Create a pydantic object describing the output format we need
class Paragraph(BaseModel):
    original_paragraph: str = Field(description="The original paragraph")
    edited_paragraph: str = Field(description="The improved edited paragraph")
    feedback: str = Field(description="Constructive feedback on the original paragraph")
    score: int = Field(description="Numeric score between 1 and 10 about the original paragraph content")


In [24]:
# add the desired output format to the LLM
structured_llm = creative_llm.with_structured_output(Paragraph)

In [25]:
chain_three = (
    {"article": lambda x: x["article"]}
    | third_prompt
    | structured_llm
    | {
        "original_paragraph": lambda x: x.original_paragraph,
        "edited_paragraph": lambda x: x.edited_paragraph,
        "feedback": lambda x: x.feedback,
        "score": lambda x: x.score
    }
)

In [26]:
out = chain_three.invoke({"article": article})

In [27]:
out

{'original_paragraph': 'In 2022, researchers at AI21 developed Jurassic-X, an LLM-based "neuro-symbolic architecture." Neuro-symbolic refers to merging the "neural computation" of large language models (LLMs) with more traditional (i.e. symbolic) computation of code.\n\nJurassic-X used the Modular Reasoning, Knowledge, and Language (MRKL) system [3]. The researchers developed MRKL to solve the limitations of LLMs, namely:\n\n- Lack of up-to-date knowledge, whether that is the latest in AI or something as simple as today\'s date.\n- Lack of proprietary knowledge, such as internal company docs or your calendar bookings.\n- Lack of reasoning, i.e. the inability to perform operations that traditional software is good at, like running complex mathematical operations.\n- Lack of ability to generalize. Back in 2022, most LLMs had to be fine-tuned to perform well in a specific domain. This problem is still present today but far less prominent as the SotA models generalize much better and, in t

### Image Generation

In [28]:
import matplotlib.pyplot as plt
from skimage import io
from langchain_community.utilities.dalle_image_generator import DallEAPIWrapper
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda

In [29]:
image_prompt = PromptTemplate(
    input_variables=["article"],
    template=(
        "Generate a prompt with less then 500 characters to generate an image"
        "based on the following article: {article}"
    )
)

In [30]:
def generate_and_display_image(image_prompt): 
    print(image_prompt)
    image_url = DallEAPIWrapper().run(image_prompt)
    image_data = io.imread(image_url)

    plt.imshow(image_data)
    plt.axis('off')
    plt.show()

In [31]:
# wrap the generator and display func into a RunnableLambda for use with LCEL
image_gen_runnable = RunnableLambda(generate_and_display_image)

In [32]:
chain_four = (
    {"article": lambda x: x["article"]}
    | image_prompt
    | llm
    | (lambda x: x.content)
    | image_gen_runnable
)

In [33]:
chain_four.invoke({"article": article})

Create an image that visually represents the concept of neuro-symbolic AI agents. Depict a futuristic scene where a large language model (LLM) interacts with traditional code, symbolizing the integration of neural and symbolic computation. Include elements like a digital brain merging with lines of code, a web search interface, and mathematical symbols. The background should suggest a dynamic, interconnected digital landscape, illustrating the vast potential of AI agents in real-world applications. Use vibrant colors to convey innovation and technology.


BadRequestError: Error code: 400 - {'error': {'message': "Unknown parameter: 'quality'.", 'type': 'invalid_request_error', 'param': 'quality', 'code': 'unknown_parameter'}}